In [2]:
# import libraries
# any way to make this always run?
import pandas as pd
import numpy as np
import requests
import re
import os
from bs4 import BeautifulSoup
import time

# make sure we're in the right directory
os.chdir('/Users/ryan-saloma/Python Projects/football_financials')

In [49]:

def get_parsed_page(url):
    # get the page
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def get_team_name(soup):
    # get the team name
    h1 = soup.find('h1').text
    team_name = re.sub(r' \d{4} Cap Table', '', h1)
    return team_name

# define the function to get the tables
def get_tables(soup):
    # get all the tables
    tables = soup.find_all('table')
    return tables

# get the h2s of the page
def get_h2s(soup):
    h2s = soup.find_all('h2')
    # extract text and remove leading/trailing whitespace
    h2_list = [h2.text.strip() for h2 in h2s]
    # keep only the h2s that start with year
    h2_list = [h2 for h2 in h2_list if re.match(r'\d{4}', h2)]
    return h2_list

# save the tables to a list with the team name
def get_team_tables(url):
    wait_time = np.random.randint(1, 5)
    time.sleep(wait_time)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    team_name = get_team_name(soup)
    table_list = pd.read_html(page.content)
    tables = []
    for table in table_list:
        tables.append(pd.DataFrame(table))
    h2s = get_h2s(soup)
    return team_name, tables, h2s

# convert the h2 into string suitable for file name
# make everything lowercase and replace spaces with underscores
# replace / with and
# remove any extra underscores
def h2_to_str(h2):
    h2 = h2.lower()
    h2 = re.sub(r' ', '_', h2)
    h2 = re.sub(r'/', '+', h2)
    h2 = re.sub(r'_+', '_', h2)
    return h2

# get the team name
def get_team_name(soup):
    # get the team name
    h1 = soup.find('h1').text
    team_name = re.sub(r' \d{4} Cap Table', '', h1)
    return team_name

# get the available years for each team
def get_available_years(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    team_name = get_team_name(soup)
    # format: <select name="year" class="form-select form-select-sm" tabindex="0" control-id="ControlID-15">
    select = soup.find('select', {'name': 'year'})
    options = select.find_all('option')
    years = [option.text for option in options]
    return team_name, years

# handle changed team names
def handle_team_name_changes(team_name):
    match team_name:
        case 'Oakland Raiders':
            return 'Las Vegas Raiders'
        case 'San Diego Chargers':
            return 'Los Angeles Chargers'
        case 'St. Louis Rams':
            return 'Los Angeles Rams'
        case 'Washington Football Team':
            return 'Washington Commanders'
        case 'Washington Redskins':
            return 'Washington Commanders'
        case _:
            return team_name


In [ ]:
# get the team urls
df = pd.read_csv(os.getcwd() + '/data/cap_space_all_teams.csv')
team_urls = df['team_url']

# get 2024 cap data for every team
team_codes = pd.read_csv(os.getcwd() + '/data/team_codes.csv')
team_codes = team_codes.set_index('team_name')
team_codes = team_codes.to_dict()['team_code']
for url in team_urls:
    team_name, tables, headers = get_team_tables(url)
    table_names = [h2_to_str(h2) for h2 in headers]
    print(team_name)
    os.mkdir(os.getcwd() + '/data/teams/' + team_name)
    for i, table in enumerate(tables):
        table.to_csv(os.getcwd() + '/data/teams/' + team_name + '/' + team_codes[team_name] + '_' + table_names[i] + '_raw.csv', index=False)

The code below should go in a separate file.

In [67]:
# clean the tables
# change the column names to lowercase and replace spaces with underscores
# replace player_(\d+) with player
# remove empty columns
# remove empty rows
# replace - with NaN
# remove dollar signs and commas from columns that should be numeric
# convert columns to numeric
# remove extra underscores from column names
# IMPORTANT: some of these steps are dependent on the previous cleaning steps (h2_to_str)

def is_string_dtype(dtype):
    return pd.api.types.is_string_dtype(dtype)

# Function to find columns with dollar sign values
def columns_with_dollar_or_percent(df):
    # List to store columns with dollar or percent signs
    cols_with_symbols = []
    
    # Iterate over columns
    for col in df.columns:
        # Check if any value in the column contains a dollar sign or percent sign
        if df[col].astype(str).str.contains(r'\$|%').any():
            cols_with_symbols.append(col)
    
    return cols_with_symbols

def clean_table(df):

    # remove columns with names that contain 'Unnamed'
    df = df.loc[:, ~df.columns.str.contains('Unnamed')]
    
    # substitute player_(\d+) or Player_(\d+) with player
    # 11/1/24: added string type check to avoid error
    # 11/1/24 2: replaced string type check with filter for Unnamed columns
    df.columns = [re.sub(r'Player \(\d+\)|player \(\d+\)', 'player', col) for col in df.columns]
    df.columns = [col.lower().replace(' ', '_') for col in df.columns]
    df = df.dropna(axis=1, how='all')
    df = df.dropna(axis=0, how='all')
    df = df.replace('-', '')
    df.columns = [re.sub(r'_+', '_', col) for col in df.columns]
    # Add player column if it doesn't exist
    # Move this to the beginning of the function
    if ('player' not in df.columns):
        df.insert(0, 'player', np.nan)

    # Remove first instance of duplicate string group in player column
    # Example: 'Carter Michael Carter' -> 'Michael Carter'
    reformatted_column = []
    for player in df['player']:
        if type(player) == str:
            reformatted_column.append(re.sub(r'^(.*?)\s+(.*?)\s+(\1)', r'\2 \1', player))
        else:
            reformatted_column.append(player)
    # reformatted_column = [re.sub(r'^(.*?)\s+(.*?)\s+(\1)', r'\2 \1', player) for player in df['player']]
    df['player'] = reformatted_column

    # check that column is string before using str.contains
    # get columns with dollar signs and commas
    cols = columns_with_dollar_or_percent(df)
    for col in cols:
        df[col] = df[col].str.replace('$', '')
        df[col] = df[col].str.replace('%', '')
        df[col] = df[col].str.replace(',', '')
        df[col] = df[col].str.replace('(', '-')
        df[col] = df[col].str.replace(')', '')
        df[col] = pd.to_numeric(df[col])

    return df

In [ ]:
# dependencies: is_string_dtype, columns_with_dollar_or_percent, clean_table

# clean the tables for 2024
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    files = os.listdir(team_path)
    files = [file for file in files if file.endswith('.csv') and 'all_players' not in file and 'cap_totals' not in file]
    all_players = pd.DataFrame()
    for file in files:
        df = pd.read_csv(team_path + '/' + file)
        df = clean_table(df)  
        all_players = pd.concat([all_players, df])
    all_players.to_csv(team_path + '/' + team_code + '2024_all_players_cleaned.csv', index=False)

In [49]:
# dependencies: get_team_name, get_available_years

# get all of the available years for each team
df = pd.read_csv(os.getcwd() + '/data/cap_space_all_teams.csv')
team_urls = df['team_url']
team_years = pd.DataFrame(columns=['team_name', 'year'])
for url in team_urls:
    team_name, years = get_available_years(url)
    for year in years:
        team_years = pd.concat([team_years, pd.DataFrame({'team_name': team_name, 'year': year}, index=[0])])
team_years.to_csv(os.getcwd() + '/data/team_years.csv', index=False)

In [ ]:
team_years = pd.read_csv(os.getcwd() + '/data/team_years.csv')
team_urls = pd.read_csv(os.getcwd() + '/data/cap_space_all_teams.csv')
team_codes = pd.read_csv(os.getcwd() + '/data/team_codes.csv')
team_codes = team_codes.set_index('team_name')
team_codes = team_codes.to_dict()['team_code']

# for url in team_urls['team_url']:
for url in team_urls['team_url']:
    # get the year from url
    year = re.search(r'(\d{4})', url).group(1)
    year = int(year)

    # go through each year and get page until 2011
    while year > 2011:

        page = requests.get(new_url)
        soup = BeautifulSoup(page.content, 'html.parser')
        team_name = get_team_name(soup)
        team_name = handle_team_name_changes(team_name)

        # get tables as list of dataframes
        table_list = pd.read_html(page.content)
        tables = []
        for table in table_list:
            tables.append(pd.DataFrame(table))

        # get h2s of the page for table names
        h2s = get_h2s(soup)
        table_names = [h2_to_str(h2) for h2 in h2s]

        dir = os.getcwd() + '/data/teams/' + team_name + '/' + str(year)
        # check if directory exists
        if not os.path.exists(dir):
            os.mkdir(dir) # assumes that directory does not exist

        for i, table in enumerate(tables):
            file = dir + '/' + team_codes[team_name] + '_' + table_names[i] + '_raw.csv'
            # check if file exists
            if os.path.exists(file):
                continue
            # check if the format is normal or cap_totals
            if 'cap_totals' not in table_names[i]:
                cleaned_table = clean_table(table)
            else:
                table.columns = table.iloc[0]
                cleaned_table = table.iloc[1:]

            cleaned_table.to_csv(file, index=False)
        year = year - 1
        new_url = re.sub(r'\d{4}', str(year), url)

In [93]:
# combine all tables that aren't 'cap_totals' into one table for each team and year
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    years = [year for year in os.listdir(team_path) if os.path.isdir(team_path + '/' + year)]
    for year in years:
        files = os.listdir(team_path + '/' + year)
        files = [file for file in files if file.endswith('_cleaned.csv') and 'cap_totals' not in file]
        # print(f'{team_name}: {files}')
        all_players = pd.DataFrame()
        for file in files:
            df = pd.read_csv(team_path + '/' + year + '/' + file)
            # check that none of df columns contain 'Cap Maximum Summary'
            if df.columns.str.contains('Cap Maximum Summary').any():
                print(f'{file} contains Cap Maximum Summary')
                df = df.loc[:, ~df.str.contains('Cap Maximum Summary')]
                df = df.dropna(axis=1, how='all')
            all_players = pd.concat([all_players, df])
        all_players.to_csv(team_path + '/' + year + '/' + team_code + '_' + year + '_all_players_cleaned.csv', index=False)

In [95]:
# combine every all_players table into one table for each team with column for year
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    years = [year for year in os.listdir(team_path) if os.path.isdir(team_path + '/' + year)]
    all_players = pd.DataFrame()
    for year in years:
        df = pd.read_csv(team_path + '/' + year + '/' + team_code + '_' + year + '_all_players_cleaned.csv')
        df['team_name'] = team_name
        df['year'] = year
        df = df[['team_name', 'year'] + [col for col in df.columns if col not in ['team_name', 'year']]]
        all_players = pd.concat([all_players, df])
    all_players.to_csv(team_path + '/' + team_code + '_all_players_cleaned.csv', index=False)

In [97]:
# combine all of all_players_cleaned tables into one table
all_players = pd.DataFrame()
for team_name in team_codes.keys():
    team_code = team_codes[team_name]
    team_path = os.getcwd() + '/data/teams/' + team_name
    files = os.listdir(team_path)
    files = [file for file in files if file.endswith('_all_players_cleaned.csv')]
    if len(files) == 0:
        print(f'{team_name} has no all_players_cleaned.csv')
        continue
    elif len(files) > 1:
        print(f'{team_name} has more than one all_players_cleaned.csv')
        continue
    df = pd.read_csv(team_path + '/' + files[0])
    all_players = pd.concat([all_players, df])

all_players.to_csv(os.getcwd() + '/data/' + 'all_players_cleaned.csv', index=False)